In [8]:
import pandas as pd
from g4f.client import Client
import re

In [9]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")


In [10]:
def generate_summary_statistics(data):
    """
    生成資料的描述性統計量。

    Args:
        data (DataFrame): 輸入資料框。

    Returns:
        DataFrame: 描述性統計量。
    """
    return data.describe()

In [11]:
def construct_prompt(user_input, summary_stats):
    """
    根據用戶輸入與數據描述生成分析 Prompt。

    Args:
        user_input (str): 用戶的自然語言查詢。
        summary_stats (DataFrame): 數據的描述性統計量。

    Returns:
        tuple: 系統提示與用戶提示。
    """
    system_prompt =("你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些描述性統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定領域。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。")
    

    
    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據此數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt

In [12]:
def generate_report(user_input, csv_file_path):
    """
    從自然語言查詢生成分析報告。

    Args:
        user_input (str): 用戶的自然語言查詢。
        csv_file_path (str): CSV 檔案路徑。

    Returns:
        str: 生成的分析報告。
    """
    try:
        # 加載數據
        data = load_data(csv_file_path)

        # 生成統計摘要
        summary_stats = generate_summary_statistics(data)

        # 構建提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 初始化 GPT 客戶端
        client = Client()

        # GPT 請求生成
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"生成報告時發生錯誤：{e}"


In [13]:
if __name__ == "__main__":
    csv_file = ".\csv.ingredient\pera_1131223.csv"
    user_query = "分析此數據並給出建議。"
    report = generate_report(user_query, csv_file)
    print(report)


根據您提供的描述性統計數據，我們可以進行以下分析：

### 1. 整體特性分析
- **數據計數 (count)**: 這裡的數據計數為3，表示我們有3個觀察值。
- **唯一值 (unique)**: 3，這意味著所有的觀察值都是獨特的，沒有重複的數據。
- **最常見的資料日期 (top)**: 資料日期為113/12/23，這是唯一的日期，顯示我們的數據集中只有一個日期的數據。
- **頻率 (freq)**: 1，表示該日期的數據出現一次。

由於數據集非常小且只有一個日期的數據，無法計算平均值、標準差或數據分佈等統計指標。

### 2. 異常值或數據問題
由於數據集僅包含3個觀察值，且每個觀察值都是獨特的，因此無法進行異常值檢測。這樣的小數據集可能會導致分析結果不具代表性，並且無法得出可靠的結論。

### 3. 類別型數據分析
在這個數據集中，類別型數據的分析受到限制，因為我們只有一個日期的數據。無法評估類別型數據的分佈特性或相關性。

### 4. 業務決策建議
- **數據收集**: 建議擴大數據集，收集更多日期的數據，以便進行更全面的分析。這將有助於計算平均值、標準差等統計指標，並能夠識別趨勢和模式。
- **關注指標**: 在收集更多數據時，應特別關注本益比、殖利率和股價淨值比的變化，這些指標對於評估股票的投資價值至關重要。

### 5. 總結與下一步建議
目前的數據集過於有限，無法進行深入分析。建議下一步行動是收集更多的數據，特別是不同日期的本益比、殖利率和股價淨值比，以便進行更全面的描述性統計分析和趨勢評估。這將有助於做出更明智的業務決策。
